In [21]:
from playwright.sync_api import sync_playwright

def scrape_economic_calendar():
    with sync_playwright() as p:
        browser = p.chromium.launch(headless=True)  # 브라우저가 보이지 않도록 설정
        page = browser.new_page()
        page.goto('https://www.dailyfx.com/economic-calendar#today')
        page.wait_for_selector('table')

        # 캘린더의 데이터 추출
        events = page.query_selector_all('table > tbody > tr')
        calendar_data = []
        for event in events:
            time = event.query_selector('td:nth-child(1)').text_content().strip()
            currency = event.query_selector('td:nth-child(2)').text_content().strip()
            event_desc = event.query_selector('td:nth-child(3)').text_content().strip()
            importance = event.query_selector('td:nth-child(4) > span').get_attribute('title')
            actual = event.query_selector('td:nth-child(5)').text_content().strip()
            forecast = event.query_selector('td:nth-child(6)').text_content().strip()
            previous = event.query_selector('td:nth-child(7)').text_content().strip()

            calendar_data.append({
                'time': time,
                'currency': currency,
                'event': event_desc,
                'importance': importance,
                'actual': actual,
                'forecast': forecast,
                'previous': previous
            })

        browser.close()
        return calendar_data

# 스크래핑 함수 실행
economic_calendar_data = scrape_economic_calendar()
print(economic_calendar_data[:5])  # 첫 5개 항목 출력


Error: It looks like you are using Playwright Sync API inside the asyncio loop.
Please use the Async API instead.

In [23]:
import tkinter as tk
from tkinter import ttk
from data.countries import regions
import yaml

def load_default_countries(filename):
    with open(filename, 'r') as file:
        data = yaml.safe_load(file)
    return data['default_selected_countries']

class CountryChecklistApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Country Checklist")
        self.regions = regions
        self.default_selected_countries = load_default_countries('settings.yaml')
        self.checkboxes = {}
        self.vars = {}
        self.max_label_width = self.calculate_max_label_width()

        self.create_checkboxes()

    def calculate_max_label_width(self):
        # 모든 지역 라벨의 너비를 가장 긴 지역 이름에 맞춥니다.
        label_width = max(len(region) for region in self.regions.keys())
        return label_width

    def create_checkboxes(self):
        for region, countries in self.regions.items():
            # 지역별 프레임 생성
            region_frame = ttk.Frame(self.root)
            region_frame.pack(side='top', fill='x', expand=True, padx=10, pady=5)

            # 지역 라벨 생성, 너비를 가장 긴 지역 이름에 맞춥니다.
            label = ttk.Label(region_frame, text=region, width=self.max_label_width, anchor='w')
            label.pack(side='left', padx=5)

            # 국가 체크박스 프레임
            countries_frame = ttk.Frame(region_frame)
            countries_frame.pack(side='left', fill='x', expand=True, padx=5)

            # 체크박스 생성
            for country in countries:
                is_selected = country.split(sep=' ')[1] in self.default_selected_countries
                var = tk.BooleanVar(value=is_selected)
                cb = ttk.Checkbutton(countries_frame, text=country, variable=var)
                cb.pack(side='left', padx=2, pady=2)
                self.checkboxes[country] = cb
                self.vars[country] = var

            # 지역별 구분선 생성
            separator = ttk.Separator(self.root, orient='horizontal')
            separator.pack(side='top', fill='x', padx=10, pady=5)

    def get_selected_countries(self):
        return [country for country, var in self.vars.items() if var.get()]

# 메인 윈도우 생성 및 앱 실행
root = tk.Tk()
app = CountryChecklistApp(root)
root.mainloop()
